# STEP15, 16: 複雑な計算グラフ

- STEP1:これまでDeZeroの変数と関数を作った
- STEP2:関数としてSquareを作った
- STEP3:別の新しい関数を実装し複数の関数を組み合わせて計算を行う
- STEP4:数値微分でいったん微分を計算してみる
- STEP5:バックプロパゲーションの仕組み
- STEP6:VariableとFunctionクラスを拡張して、バックプロパゲーションを用いて微分を求められるように実装
- STEP7:順伝搬がどのような計算であっても自動的に逆伝搬を計算できるようにする, 具体的にはVariableクラスを拡張し使用した関数情報を保持できるようにする
- STEP8:処理効率の改善するために、backwardメソッドをwhileループに置き換える。Variable関数のみの書き換えでOK
- STEP9:pythonの関数として使えるようにする, y.grad=np.array(1.0)を省略する, ndarrayだけ扱う
- STEP10:DeepLearningのフレームワークのテスト方法について説明
- STEP11:関数に対して可変長入出力に対応する
- STEP12:11の拡張
- STEP13:逆伝搬に関しても関数に対して可変長入出力に対応する
- STEP14:同じ変数を繰り返し使うと発生する問題に対応する y = add(x, x)
- STEP15:さまざまなトポロジーの計算グラフに対応すること(説明)
- STEP16:さまざまなトポロジーの計算グラフに対応すること(実装)

In [1]:
#DIR = "deep-learning-from-scratch-3/steps/"
#! diff $DIR/step14.py $DIR/step16.py -y

## STEP15, 16の説明で使うモデル

![](https://docs.google.com/drawings/d/e/2PACX-1vQbIDXMxsjyJH2qgyFbi_SRQnuhhBZg-sL4KNTVo07WOOEoqWbgMTZO8Pw1mIIk-uf-4Y_l-QO7qRC6/pub?w=960&h=779)

## STEP14のコードを用いて課題を確認する

### STEP14 ソースコード

In [2]:
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        #icount = 0 # added by Daisuke Nagao
        while funcs:
            #print(f"---- while {icount}回目 ----") # added by Daisuke Nagao
            #print(f"funcs_before = {funcs}") # added by Daisuke Nagao
            f = funcs.pop()
            #print(f"funcs_after  = {funcs}") # added by Daisuke Nagao
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    funcs.append(x.creator)
            #icount += 1 # added by Daisuke Nagao


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

def square(x):
    return Square()(x)

class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)

In [3]:
# わかりやすさのため、一旦、下記をコメントアウト, 使用する関数のClassを指定する方法でテストを実行する
#x = Variable(np.array(2))
#a = square(x); print("a:", a.data)
#b = square(a); print("b:", b.data)
#c = square(a); print("c:", c.data)
#y = add(b ,c); print("y:", y.data)
#y.backward()

A = Square()
B = Square()
C = Square()
D = Add()

x = Variable(np.array(2))
a = A(x) 
b = B(a)                 
c = C(a)                 
y = D(b, c)

print("x:", x.data)
print("a:", a.data)
print("b:", b.data)
print("c:", c.data)
print("y:", y.data)

y.backward()

print("y.grad", y.grad)
print("c.grad", c.grad)
print("b.grad", b.grad)
print("a.grad", a.grad)
print("x.grad", x.grad)

x: 2
a: 4
b: 16
c: 16
y: 32
y.grad 1
c.grad 1
b.grad 1
a.grad 16
x.grad 96


### ソースコードのコメントアウトを外して上記のテストをもう一度行う

オブジェクトのhashは下記を実行し一致するものを探す

In [4]:
print("A:", A)
print("B:", B)
print("C:", C)
print("D:", D, D.__hash__())

A: <__main__.Square object at 0x10421c190>
B: <__main__.Square object at 0x10421c520>
C: <__main__.Square object at 0x10421c1f0>
D: <__main__.Add object at 0x10421c2b0> 272768043


## STEP16: 改善版

### STEP16ソースコード

In [5]:
import numpy as np

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        #icount = 0 # added by Daisuke Nagao
        while funcs:
            #print(f"---- while {icount}回目 ----") # added by Daisuke Nagao
            #print(f"funcs_before = {funcs}") # added by Daisuke Nagao
            f = funcs.pop()
            #print(f"funcs_after  = {funcs}") # added by Daisuke Nagao
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)
            #icount += 1 # added by Daisuke Nagao


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)

In [6]:
# わかりやすさのため、一旦、下記をコメントアウト, 使用する関数のClassを指定する方法でテストを実行する
#x = Variable(np.array(2))
#a = square(x); print("a:", a.data)
#b = square(a); print("b:", b.data)
#c = square(a); print("c:", c.data)
#y = add(b ,c); print("y:", y.data)
#y.backward()

A = Square()
B = Square()
C = Square()
D = Add()

x = Variable(np.array(2))
a = A(x) 
b = B(a)                 
c = C(a)                 
y = D(b, c)

print("x:", x.data)
print("a:", a.data)
print("b:", b.data)
print("c:", c.data)
print("y:", y.data)

y.backward()

print("y.grad", y.grad)
print("c.grad", c.grad)
print("b.grad", b.grad)
print("a.grad", a.grad)
print("x.grad", x.grad)

x: 2
a: 4
b: 16
c: 16
y: 32
y.grad 1
c.grad 1
b.grad 1
a.grad 16
x.grad 64


### ソースコードのコメントアウトを外して上記のテストをもう一度行う

In [7]:
print("A:", A)
print("B:", B)
print("C:", C)
print("D:", D)

A: <__main__.Square object at 0x10421cf10>
B: <__main__.Square object at 0x10421ce50>
C: <__main__.Square object at 0x10421c3a0>
D: <__main__.Add object at 0x10421ccd0>
